In [ ]:
# Load core libraries for data handling, evaluation, and plotting

import numpy as np               # numerical computing & array handling
import matplotlib.pyplot as plt  # for creating line charts and visualisations
from sklearn.metrics import mean_squared_error  # to evaluate prediction accuracy
from math import sqrt             # optional: for manual RMSE calculation

# Step 1: Set up the Keras components needed to build the LSTM model

from keras.models import Sequential   # model container to stack layers sequentially
from keras.layers import LSTM, Dense  # LSTM for sequence learning, Dense for output layer

# Step 2: Import prepared datasets for training, validation, and testing
# Each CSV contains weather predictor variables and the corresponding solar irradiance target.
# Paths are written as relative so the notebook runs on any system without modification.

train_data = np.loadtxt("train_solar_data.csv", delimiter=",")
validate_data = np.loadtxt("validate_solar_data.csv", delimiter=",")
test_data = np.loadtxt("test_solar_data.csv", delimiter=",")

# Step 3: Separate features (X) from target values (y)
# The first 9 columns store weather-related predictors, while the final column holds the solar irradiance.

x_tr, t_tr = train_data[:, :9],  train_data[:, -1]  # training set
x_va, t_va = validate_data[:, :9], validate_data[:, -1]  # validation set
x_te, t_te = test_data[:, :9],    test_data[:, -1]  # test set

# Step 4: Count complete days in each dataset
HOURS_PER_DAY = 11  # number of time steps per day

Ndays_tr = len(x_tr) // HOURS_PER_DAY
Ndays_va = len(x_va) // HOURS_PER_DAY
Ndays_te = len(x_te) // HOURS_PER_DAY

# Step 5: Reshape for LSTM input
# Format: [days, time steps per day, features]

train_x = x_tr.reshape(Ndays_tr, HOURS_PER_DAY, 9)
train_t = t_tr.reshape(Ndays_tr, HOURS_PER_DAY, 1)

validate_x = x_va.reshape(Ndays_va, HOURS_PER_DAY, 9)
validate_t = t_va.reshape(Ndays_va, HOURS_PER_DAY, 1)

test_x = x_te.reshape(Ndays_te, HOURS_PER_DAY, 9)
test_t = t_te.reshape(Ndays_te, HOURS_PER_DAY, 1)

# Step 6: Build the LSTM model for day-ahead solar forecasting

# Start a simple sequential model
model = Sequential()

# LSTM layer with 50 units; processes daily sequences of weather features
model.add(LSTM(50, input_shape=(HOURS_PER_DAY, 9), return_sequences=True))

# Output layer: predicts a continuous value (solar irradiance) for each time step
model.add(Dense(1, activation="linear"))

# Compile model with mean squared error loss and Adam optimiser
model.compile(loss="mse", optimizer="adam")

# Step 7: Train the model

# Fit on training data for 100 epochs, using validation set to monitor performance
history = model.fit(
    train_x, train_t,
    epochs=100,
    batch_size=50,
    validation_data=(validate_x, validate_t)
)

# Step 8: Plot training loss over epochs

plt.plot(history.history["loss"], label="Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Mean Squared Error")
plt.title("Training Loss")
plt.legend()
plt.show()

# Step 9: Predict solar irradiance on the test set
yhat = model.predict(test_x)

# Step 10: Flatten predictions for comparison
y_te = yhat.reshape(Ndays_te * HOURS_PER_DAY,)

# Step 11: Calculate RMSE (Root Mean Squared Error) for test accuracy
# Lower RMSE means better predictions; scaling matches the original study's method

rmse2 = mean_squared_error(y_te, t_te) * Ndays_te * HOURS_PER_DAY
rmse = sqrt(rmse2 / 4026) * 1087.4396 / 2
print(f"Test RMSE: {rmse:.3f}")

Epoch 1/100


/opt/anaconda3/envs/ml311/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.2401 - val_loss: 0.1137
Epoch 2/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.1027 - val_loss: 0.0698
Epoch 3/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0627 - val_loss: 0.0461
Epoch 4/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0447 - val_loss: 0.0401
Epoch 5/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0376 - val_loss: 0.0328
Epoch 6/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0327 - val_loss: 0.0333
Epoch 7/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0309 - val_loss: 0.0302
Epoch 8/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0286 - val_loss: 0.0279
Epoch 9/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0286 - val_loss: 0.0269
Epoch 10/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0260 - val_loss: 0.0271
Epoch 11/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0260 - val_loss: 0.0261
Epoch 12/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0245 